In [48]:
import numpy as np

In [26]:
from gensim.corpora import TextCorpus, MmCorpus
from gensim import utils

# override corpus with your own tokenizer for trial/error
list(utils.tokenize("energetic outdoors wilderness independence alone"))

corpus = TextCorpus('./descriptions')
corpus.dictionary.filter_extremes(no_below=3, no_above=.9, keep_n=10000)

In [24]:
# Create TFIDF
from gensim.models.tfidfmodel import TfidfModel
tfidf = TfidfModel(corpus)

In [25]:
# get the BOW (TF) for a doc
tf = corpus.dictionary.doc2bow(utils.tokenize("energetic outdoors wilderness independence alone"))
tfidf[tf]

[]

In [13]:
from gensim.models.word2vec import Word2Vec

model = Word2Vec.load("profiles.model")

In [218]:
def sample(w, size = 50, pwr = 1.75):
    t,f = w[:,0], w[:,1]**pwr
    p = f/np.linalg.norm(f, 1)
    return np.random.choice(t, size = size, replace = True, p = p)

def doc_vec(w, model, corpus):
    # TODO: deal with things not in vocabulary.... 
    vecs = [model[corpus.dictionary[x]] for x in sample(w)]
    return np.mean(vecs, 0)    

In [15]:
sentences = [list(g) for g in list(corpus.get_texts())]
doc = sentences[2]
w = np.array(tfidf[corpus.dictionary.doc2bow(doc)])
m = doc_vec(w, model, corpus)
word.similar_by_vector(m)

NameError: name 'corpus' is not defined

In [166]:
' '.join(sentences[2])

u'measurement manager for measurement campaigns on our offshore wind farmsdo you have excellent project management skills join us and become measurement manager in the measurements department together with your colleagues you will work closely with suppliers of wind sea state and structural load measurement equipment to ensure the best technologies for deployment at our wind farms we also work closely with stakeholders across the organisation including experts in wind and sea state assessment load analysis foundation design wtg technology and site classification on designing the best measurement campaigns for our offshore wind farms as part of the measurements team which counts employees you will manage projects for designing and implementing wind wave structural load and power curve measurement programs for utility scale wind farms you will be project manager for a team of specialists covering several highly specialised areas of expertise measurements is part of wind power which has e

In [ ]:
from gensim import utils
from gensim.models.word2vec import LineSentence
import itertools
from gensim.models.word2vec_inner import FAST_VERSION, MAX_WORDS_IN_BATCH

class TokenSentence(object):
    """
    Simple format: one sentence = one line; words already preprocessed and separated by whitespace.
    """

    def __init__(self, source, max_sentence_length=MAX_WORDS_IN_BATCH, limit=None, preprocess=lambda x: utils.to_unicode(x).split()):
        """
        `source` can be either a list or single item which is either a string or a file object. Clip the file to the first
        `limit` lines (or no clipped if limit is None, the default).
        `preprocess` is a function that takes one argument, a string, and returns a list of tokens (defaults to utils.to_unicode(line).split())
        Example::
            sentences = LineSentence('myfile.txt')
        Or for compressed files::
            sentences = LineSentence('compressed_text.txt.bz2')
            sentences = LineSentence('compressed_text.txt.gz')
        """
        self.sources = source if type(source) == list else [source]
        self.max_sentence_length = max_sentence_length
        self.limit = limit
        self.preprocess = preprocess

    def __iter__(self):
        """Iterate through the lines in the source."""
        def process(source):
            for line in itertools.islice(source, self.limit):
                line = self.preprocess(line)
                i = 0
                while i < len(line):
                    yield line[i : i + self.max_sentence_length]
                    i += self.max_sentence_length

        try:
            # Assume it is a file-like object and try treating it as such
            # Things that don't have seek will trigger an exception            
            self.sources[0].seek(0)
            for s in self.sources:
                for i in process(s):
                    yield i
            
        except AttributeError:
            # If it didn't work like a file, use it as a string filename
            for s in self.sources:
                with utils.smart_open(s) as fin:
                    for i in process(fin):
                        yield i

tokenize = lambda x: list(utils.tokenize(x, lower=True, deacc=True, errors='ignore'))
a = TokenSentence(['descriptions', 'descriptions'], preprocess = tokenize)
a = list(itertools.islice((i for i in a), 1400, 1420))
len(a)
a[3]

In [325]:
from scipy.stats import gaussian_kde

# gaussian_kde(word.wv)

kde = gaussian_kde(word.wv.syn0norm)

In [ ]:
size = 10
# np.random.choice(word.wv.syn0norm, size = size)
X = word.wv.syn0norm
X[np.random.choice(X.shape[0], size = 10, replace=False), :]